In [ ]:
!pip install datasets
!pip install peft
# !pip install unsloth
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.26" trl peft accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 6.4 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 8.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-m

In [ ]:
from datasets import load_dataset, load_from_disk, concatenate_datasets, Dataset
import pandas as pd
import os
import random
import functools
import csv
import numpy as np
import torch
import torch.nn.functional as F
from sklearn.metrics import f1_score, accuracy_score
# from skmultilearn.model_selection import iterative_train_test_split
from datasets import Dataset, DatasetDict
from peft import (
    LoraConfig,
    prepare_model_for_kbit_training,
    get_peft_model,
    PeftModel
)
from transformers import (
    AutoModelForSequenceClassification,
    MistralForSequenceClassification,
    PretrainedConfig,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    AutoModel
)
# from unsloth import FastLanguageModel

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dataset_location = "/content/drive/MyDrive/685 Final Project/Datasets/medmcqa-prompts"

In [ ]:
train_dataset = load_from_disk(f"{dataset_location}/train_prompts.hf")
test_dataset = load_from_disk(f"{dataset_location}/test_prompts.hf")
eval_dataset = load_from_disk(f"{dataset_location}/eval_prompts.hf")

# train_dataset = load_from_disk(f"{dataset_location}/train_prompts_micro.hf")
# test_dataset = load_from_disk(f"{dataset_location}/test_prompts_micro.hf")
# eval_dataset = load_from_disk(f"{dataset_location}/eval_prompts_micro.hf")

# train_dataset = load_from_disk(f"{dataset_location}/train_prompts_mini.hf")
# test_dataset = load_from_disk(f"{dataset_location}/test_prompts_mini.hf")
# eval_dataset = load_from_disk(f"{dataset_location}/eval_prompts_mini.hf")

In [ ]:
train_dataset_mini = load_from_disk(f"{dataset_location}/train_prompts_mini.hf")
# test_dataset = load_from_disk(f"{dataset_location}/test_prompts_mini.hf")
# eval_dataset = load_from_disk(f"{dataset_location}/eval_prompts_mini.hf")

In [ ]:
set(train_dataset['cop'])

{0, 1, 2, 3}

In [ ]:
num_classes = len(set(train_dataset['cop']))
num_classes

4

In [ ]:
train_dataset

Dataset({
    features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name', 'prompt', 'label_one_hot'],
    num_rows: 182822
})

In [ ]:
eval_dataset

Dataset({
    features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name', 'prompt', 'label_one_hot'],
    num_rows: 4183
})

In [ ]:
# preprocess dataset with tokenizer
def tokenize_examples(examples, tokenizer):
    tokenized_inputs = tokenizer(examples['prompt'])
    # tokenized_inputs['labels'] = examples['cop']
    tokenized_inputs['labels'] = examples['label_one_hot']
    # print(f"examples['cop']: {examples['cop']}")
    return tokenized_inputs

# Mistral SFT

In [ ]:
### First Load the Base Model

model_name = "unsloth/mistral-7b-instruct-v0.2-bnb-4bit"

# qunatization config
quantization_config = BitsAndBytesConfig(
    load_in_4bit = True, # enable 4-bit quantization
    bnb_4bit_quant_type = 'nf4', # information theoretically optimal dtype for normally distributed weights
    bnb_4bit_use_double_quant = True, # quantize quantized weights //insert xzibit meme
    bnb_4bit_compute_dtype = torch.bfloat16 # optimized fp format for ML
)


model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    num_labels= num_classes,
)

Unused kwargs: ['quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/usr/local/lib/python3.10/dist-packages/transformers/quantizers/auto.py:159: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)
`low_cpu_mem_usage` was None, now set to True since model is quantized.
Some weights of MistralForSequenceClassification were not initialized from the model checkpoint at unsloth/mistral-7b-instruct-v0.2-bnb-4bit and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
saved_model_location = "/content/drive/MyDrive/685 Final Project/Models"

In [ ]:
trained_model_id = "Mistral_MedMCQ_Mini_dataset"

In [ ]:
# Loading Trained Model

model = PeftModel.from_pretrained(model = model, model_id = f"{saved_model_location}/{trained_model_id}")
tokenizer= AutoTokenizer.from_pretrained(model_name) # tokenizer of unsloth



tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
tokenizer

LlamaTokenizerFast(name_or_path='unsloth/mistral-7b-instruct-v0.2-bnb-4bit', vocab_size=32000, model_max_length=32768, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [ ]:
# ds = DatasetDict({
#     'train': Dataset.from_dict({'text': x_train, 'labels': y_train}),
#     'val': Dataset.from_dict({'text': x_val, 'labels': y_val})
# })

tokenized_train = train_dataset.map(functools.partial(tokenize_examples, tokenizer=tokenizer), batched=True)
tokenized_train = tokenized_train.with_format('torch')
tokenized_train[0]

{'id': 'e9ad821a-c438-4965-9f77-760819dfa155',
 'question': 'Chronic urethral obstruction due to benign prismatic hyperplasia can lead to the following change in kidney parenchyma',
 'opa': 'Hyperplasia',
 'opb': 'Hyperophy',
 'opc': 'Atrophy',
 'opd': 'Dyplasia',
 'cop': tensor(2),
 'choice_type': 'single',
 'exp': 'Chronic urethral obstruction because of urinary calculi, prostatic hyperophy, tumors, normal pregnancy, tumors, uterine prolapse or functional disorders cause hydronephrosis which by definition is used to describe dilatation of renal pelvis and calculus associated with progressive atrophy of the kidney due to obstruction to the outflow of urine Refer Robbins 7yh/9,1012,9/e. P950',
 'subject_name': 'Anatomy',
 'topic_name': 'Urinary tract',
 'prompt': '\n    Question:\n    Chronic urethral obstruction due to benign prismatic hyperplasia can lead to the following change in kidney parenchyma\nOptions:\nA. Hyperplasia\nB. Hyperophy\nC. Atrophy\nD. Dyplasia\n\n    [INST] Solve 

In [ ]:
# tokenized_train[0]

In [ ]:
tokenized_eval = eval_dataset.map(functools.partial(tokenize_examples, tokenizer=tokenizer), batched=True)
tokenized_eval = tokenized_eval.with_format('torch')
tokenized_eval

Dataset({
    features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name', 'prompt', 'label_one_hot', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 4183
})

In [ ]:
tokenized_train['labels']

tensor([[0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        ...,
        [0., 0., 1., 0.],
        [1., 0., 0., 0.],
        [0., 0., 0., 1.]])

In [ ]:
tokenized_train['prompt'][0]

'\n    Question:\n    Chronic urethral obstruction due to benign prismatic hyperplasia can lead to the following change in kidney parenchyma\nOptions:\nA. Hyperplasia\nB. Hyperophy\nC. Atrophy\nD. Dyplasia\n\n    [INST] Solve this Multiple Choice Question and provide the correct option out of four options(A,B,C,D) as the answer. [/INST]\n    Answer: </s>'

In [ ]:
tokenized_train[0]

{'id': 'e9ad821a-c438-4965-9f77-760819dfa155',
 'question': 'Chronic urethral obstruction due to benign prismatic hyperplasia can lead to the following change in kidney parenchyma',
 'opa': 'Hyperplasia',
 'opb': 'Hyperophy',
 'opc': 'Atrophy',
 'opd': 'Dyplasia',
 'cop': tensor(2),
 'choice_type': 'single',
 'exp': 'Chronic urethral obstruction because of urinary calculi, prostatic hyperophy, tumors, normal pregnancy, tumors, uterine prolapse or functional disorders cause hydronephrosis which by definition is used to describe dilatation of renal pelvis and calculus associated with progressive atrophy of the kidney due to obstruction to the outflow of urine Refer Robbins 7yh/9,1012,9/e. P950',
 'subject_name': 'Anatomy',
 'topic_name': 'Urinary tract',
 'prompt': '\n    Question:\n    Chronic urethral obstruction due to benign prismatic hyperplasia can lead to the following change in kidney parenchyma\nOptions:\nA. Hyperplasia\nB. Hyperophy\nC. Atrophy\nD. Dyplasia\n\n    [INST] Solve 

In [ ]:
tokenized_train['input_ids'][0].shape, tokenized_train['input_ids'][1].shape

(torch.Size([114]), torch.Size([93]))

In [ ]:
tokenized_eval['labels'][4]

tensor([1., 0., 0., 0.])

In [ ]:
train_dataset[2]

{'id': '5c38bea6-787a-44a9-b2df-88f4218ab914',
 'question': 'All of the following are surgical options for morbid obesity except -',
 'opa': 'Adjustable gastric banding',
 'opb': 'Biliopancreatic diversion',
 'opc': 'Duodenal Switch',
 'opd': 'Roux en Y Duodenal By pass',
 'cop': 3,
 'choice_type': 'multi',
 'exp': "Ans. is 'd' i.e., Roux en Y Duodenal Bypass Bariatric surgical procedures include:a. Vertical banded gastroplastyb. Adjustable gastric bandingc. Roux-en Y gastric bypass (Not - Roux-en Y Duodenal Bypass)d. Biliopancreatic diversione. Duodenal switcho The surgical treatment of morbid obesity is known as bariatric surgery.o Morbid obesity is defined as body mass index of 35 kg/m2 or more with obesity related comorbidity, or BMI of 40 kg/m2 or greater without comorbidity.o Bariatric operations produce weight loss as a result of 2 factors. One is restriction of oralintake. The other is malabsorbtion of ingested food.o Gastric restrictive procedures include Vertical banded gastr

In [ ]:
train_dataset[2]['prompt']


'\n    Question:\n    All of the following are surgical options for morbid obesity except -\nOptions:\nA. Adjustable gastric banding\nB. Biliopancreatic diversion\nC. Duodenal Switch\nD. Roux en Y Duodenal By pass\n\n    [INST] Solve this Multiple Choice Question and provide the correct option out of four options(A,B,C,D) as the answer. [/INST]\n    Answer: </s>'

In [ ]:
tokenized_eval.select(range(4))["prompt"]

['\n    Question:\n    Which of the following is not true for myelinated nerve fibers:\nOptions:\nA. Impulse through myelinated fibers is slower than non-myelinated fibers\nB. Membrane currents are generated at nodes of Ranvier\nC. Saltatory conduction of impulses is seen\nD. Local anesthesia is effective only when the nerve is not covered by myelin sheath\n\n    [INST] Solve this Multiple Choice Question and provide the correct option out of four options(A,B,C,D) as the answer. [/INST]\n    Answer: </s>',
 "\n    Question:\n    Which of the following is not true about glomerular capillaries')\nOptions:\nA. The oncotic pressure of the fluid leaving the capillaries is less than that of fluid entering it\nB. Glucose concentration in the capillaries is the same as that in glomerular filtrate\nC. Constriction of afferent aeriole decreases the blood flow to the glomerulas\nD. Hematocrit of the fluid leaving the capillaries is less than that of the fluid entering it\n\n    [INST] Solve this 

In [ ]:
inputs = tokenizer(
[
    # "\n    Question:\n    Chronic urethral obstruction due to benign prismatic hyperplasia can lead to the following change in kidney parenchyma\nOptions:\nA. Hyperplasia\nB. Hyperophy\nC. Atrophy\nD. Dyplasia\n\n    [INST] Solve this Multiple Choice Question and provide the correct option out of four options(A,B,C,D) as the answer. [/INST]\n    Answer: "
    # ,
    train_dataset[1]['prompt'],
    # train_dataset[2]['prompt']
], return_tensors = "pt",
).to("cuda")
outputs = model.forward(**inputs)
# tokenizer.batch_decode(outputs)

In [ ]:
tokenizer(
[
    # "\n    Question:\n    Chronic urethral obstruction due to benign prismatic hyperplasia can lead to the following change in kidney parenchyma\nOptions:\nA. Hyperplasia\nB. Hyperophy\nC. Atrophy\nD. Dyplasia\n\n    [INST] Solve this Multiple Choice Question and provide the correct option out of four options(A,B,C,D) as the answer. [/INST]\n    Answer: "
    # ,
    train_dataset[1]['prompt'],
    # train_dataset[2]['prompt']
])

{'input_ids': [[1, 28705, 13, 2287, 22478, 28747, 13, 2287, 9595, 11781, 8255, 349, 16584, 477, 865, 8527, 2832, 28747, 13, 4018, 28747, 13, 28741, 28723, 20874, 8255, 334, 13, 28760, 28723, 20874, 8255, 365, 28787, 13, 28743, 28723, 20874, 8255, 365, 28740, 28750, 13, 28757, 28723, 20874, 8255, 384, 13, 13, 2287, 733, 16289, 28793, 318, 5303, 456, 9713, 4191, 27710, 22478, 304, 3084, 272, 4714, 3551, 575, 302, 2308, 2877, 28732, 28741, 28725, 28760, 28725, 28743, 28725, 28757, 28731, 390, 272, 4372, 28723, 733, 28748, 16289, 28793, 13, 2287, 26307, 28747, 28705, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [ ]:
def collate_fn(batch, tokenizer):
## Adds Padding from the tokenizer.
    dict_keys = ['input_ids', 'attention_mask', 'labels']
    # print(batch)
    # d = {k: [print(k, dic) for dic in batch] for k in dict_keys}
    d = {k: [dic[k] for dic in batch] for k in dict_keys}
    d['input_ids'] = torch.nn.utils.rnn.pad_sequence(
        d['input_ids'], batch_first=True, padding_value=tokenizer.pad_token_id
    )
    d['attention_mask'] = torch.nn.utils.rnn.pad_sequence(
        d['attention_mask'], batch_first=True, padding_value=0
    )
    d['labels'] = torch.stack(d['labels'])
    # d['cop'] = torch.stack(d['cop'])

    return d

In [ ]:
tokenized_train.select(range(2))

Dataset({
    features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name', 'prompt', 'label_one_hot', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 2
})

In [ ]:
collator = functools.partial(collate_fn, tokenizer=tokenizer)
collator(tokenized_train.select(range(2)))

{'input_ids': tensor([[    1, 28705,    13,  2287, 22478, 28747,    13,  2287,   689,  7624,
            332,   267,   362,  1650, 13364,  3112,  2940,   298,  2798,   603,
            724,  1443,  1711, 11503,   452, 28038,   541,  1736,   298,   272,
           2296,  2268,   297,  6371,  2705,   284,  4184,   338, 28724,   705,
             13,  4018, 28747,    13, 28741, 28723, 10540,   487,   452, 28038,
             13, 28760, 28723, 10540,   487,  3126, 28724,    13, 28743, 28723,
           1794,   311,  8071,    13, 28757, 28723, 25685,   452, 28038,    13,
             13,  2287,   733, 16289, 28793,   318,  5303,   456,  9713,  4191,
          27710, 22478,   304,  3084,   272,  4714,  3551,   575,   302,  2308,
           2877, 28732, 28741, 28725, 28760, 28725, 28743, 28725, 28757, 28731,
            390,   272,  4372, 28723,   733, 28748, 16289, 28793,    13,  2287,
          26307, 28747, 28705,     2],
         [    1, 28705,    13,  2287, 22478, 28747,    13,  2287,  9

In [ ]:
d = dict(collator(tokenized_train.select(range(2))))

In [ ]:
model.eval()

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): MistralForSequenceClassification(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lor

In [ ]:
outputs = model.forward(**d)

In [ ]:
outputs.logits

tensor([[-5.1250, -2.0312, -8.2344, -2.6602],
        [-0.8384,  7.7617, -3.9805,  3.3555]], dtype=torch.float16,
       grad_fn=<ToCopyBackward0>)

In [ ]:
torch.argmax(outputs.logits, dim=-1)

tensor([1, 1])

In [ ]:
labels_df = pd.DataFrame(tokenized_eval['cop'], columns=['true'])

In [ ]:
labels_df.head()

,true
0,0
1,0
2,2
3,2
4,0


In [ ]:
mistral_labels = []
batch_size = 16

for i in range(0, len(tokenized_eval), batch_size):
  print(i)
# for i in range(20, 30, batch_size):

  # print(i)
  batch = tokenized_eval.select(range(i, min(i + batch_size, len(tokenized_eval))))
  # print(batch)
  # batch = collate_fn(batch, tokenizer)
  # batch
  collated_batch = dict(collator(batch))
  outputs = model.forward(**collated_batch)
  outputs_logits = torch.argmax(outputs.logits, dim=-1)
  # print(outputs.logits)
  mistral_labels.extend(outputs_logits)

  # print(batch)


0
16
32
48
64
80
96
112
128
144
160
176
192
208
224
240
256
272
288
304
320
336
352
368
384
400
416
432
448
464
480
496
512
528
544
560
576
592
608
624
640
656
672
688
704
720
736
752
768
784
800
816
832
848
864
880
896
912
928
944
960
976
992
1008
1024
1040
1056
1072
1088
1104
1120
1136
1152
1168
1184
1200
1216
1232
1248
1264
1280
1296
1312
1328
1344
1360
1376
1392
1408
1424
1440
1456
1472
1488
1504
1520
1536
1552
1568
1584
1600
1616
1632
1648
1664
1680
1696
1712
1728
1744
1760
1776
1792
1808
1824
1840
1856
1872
1888
1904
1920
1936
1952
1968
1984
2000
2016
2032
2048
2064
2080
2096
2112
2128
2144
2160
2176
2192
2208
2224
2240
2256
2272
2288
2304
2320
2336
2352
2368
2384
2400
2416
2432
2448
2464
2480
2496
2512
2528
2544
2560
2576
2592
2608
2624
2640
2656
2672
2688
2704
2720
2736
2752
2768
2784
2800
2816
2832
2848
2864
2880
2896
2912
2928
2944
2960
2976
2992
3008
3024
3040
3056
3072
3088
3104
3120
3136
3152
3168
3184
3200
3216
3232
3248
3264
3280
3296
3312
3328
3344
3360
3376
3392
3408
3

In [ ]:
labels_df['mistral'] = [int(x) for x in mistral_labels]

In [ ]:
labels_df['mistral'] = mistral_labels

In [ ]:
labels_df.describe()

,true,mistral
count,4183.000000,4183.000000
mean,1.293330,1.004781
std,1.116491,0.097683
min,0.000000,1.000000
25%,0.000000,1.000000
50%,1.000000,1.000000
75%,2.000000,1.000000
max,3.000000,3.000000


In [ ]:
labels_df['mistral'].value_counts()

mistral
1    4173
3      10
Name: count, dtype: int64

In [ ]:

accuracy_score(labels_df['true'], labels_df['mistral'])

0.26033946928042073

In [ ]:
labels_df[:-32]

,true,mistral
0,0,1
1,0,1
2,2,1
3,2,1
4,0,1
...,...,...
4146,0,1
4147,3,1
4148,3,1
4149,0,1


In [ ]:
accuracy_score(labels_df[:-32]['true'], labels_df[:-32]['mistral'])

0.2599373644904842

In [ ]:
output_location = "/content/drive/MyDrive/685 Final Project"


In [ ]:
labels_df.to_csv(f"{output_location}/mistral_sft.csv")